In [42]:
import pandas as pd

mini = pd.read_csv('purecot-gemma9b-wikihop.csv')
# med = pd.read_csv('cot-phimedium-wikihop-v2.csv')
# two = pd.read_csv('base-gemma-2b-wikihop.csv')
# nine = pd.read_csv('base-gemma-9b-wikihop.csv')

In [2]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
from guidance import models, select

/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


In [3]:
mini.head()

,Unnamed: 0,query,correct,thesis,antithesis,pre-synthesis,oracle,context
0,0,What event did Juan Rossell participate in?,1996 summer olympics,1996 summer olympics,The most correct answer is **1996 summer olymp...,The most correct answer is **1996 summer olymp...,1996 summer olympics,"The 2004 Summer Olympic Games, held in Athens..."
1,1,What languages did John Osteen speak or write?,english,nahuatl,The most correct answer is **english**. \n\nWh...,The most correct answer is **english**. \n\nTh...,english,Christianity is a monotheistic religion based...
2,2,What is the parent taxon of Australosuchus?,crocodilia,animal,The most correct answer is **'crocodilia'**. \...,You are absolutely correct! \n\nThe most corre...,crocodilia,Mekosuchinae was a subfamily of crocodiles th...
3,3,What is the'record_label method man' associate...,loud records,1995,The most correct answer is **'epic records'**....,You are absolutely correct! \n\nThe most corre...,loud records,"Wu-Tang Clan's debut album, ""Enter the Wu-Tan..."
4,4,In which administrative territorial entity is ...,raipur district,new delhi,The correct answer is **chhattisgarh**. \n\nTh...,The most correct answer is **chhattisgarh**. \...,raipur district,"Chhattisgarh, a state in central India, is ri..."


In [45]:
two = two.drop(['Unnamed: 0'], axis = 1)

In [4]:
from langchain_core.prompts import PromptTemplate
from operator import itemgetter

# prompt augmentation for the (format of the) synthesis:
prompt_template = PromptTemplate.from_template(
"""You are a multiple-choice question answering assistant.
Choose the most proper option between {options} that best matches with the suggestion. 

Question: {question}
Context: {critique}
Sources: {context}

Assistant:
"""
)
augmentation = {"question": itemgetter("question"),
                "options": itemgetter("options"), 
                "critique": itemgetter("critique"),
                "context": itemgetter("context"), }
synthesis_chain = augmentation | prompt_template 

In [5]:
import ast
import re

# Definisci una funzione di pulizia per rimuovere caratteri non validi
def clean_text(text):
    return re.sub(r"[^\w\s.,!?\-:;()]+", '', text)

def synthesisGeneration(query, merged, pre_answer, sources):
    merged = ast.literal_eval(merged)
    merged.append('not clear given the context')
    augmented_prompt = synthesis_chain.invoke({'question': query, 
                                            'options': merged,
                                            'critique': pre_answer,
                                            'context': sources})

    normal_string = clean_text(augmented_prompt.text)
    ans = new_model + normal_string + select(merged)
    return str(ans)

def extract_answer_synthesis(text):
    # Trova l'indice in cui inizia il testo "Why or why not the answer is correct:"
    start_index = text.find("\n\nAssistant:\n")

    
    # Se l'indice è stato trovato, estrai la risposta corretta
    if start_index != -1:
        start_index += len("\n\nAssistant:\n")
        # Estrai il testo dopo "Why or why not the answer is correct:"
        correct_answer_text = text[start_index:].strip()
        return correct_answer_text
    else:
        return "The correct answer could not be found."

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct", use_fast=False)
new_model = models.Transformers(model, tokenizer, temperature=0.0)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/orfeo/cephfs/home/dssc/scandu00/nlp-env/lib64/python3.9/site-packages/guidance/chat.py:73: UserWarning: Chat template {% for message in messages %}{% if message['role'] == 'system' %}{{'<|system|>
' + message['content'] + '<|end|>
'}}{% elif message['role'] == 'user' %}{{'<|user|>
' + message['content'] + '<|end|>
'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>
' + message['content'] + '<|end|>
'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>
' }}{% else %}{{ eos_token }}{% endif %} was unable to be loaded directly into guidance.
                        Defaulting to the ChatML format which may not be optimal for the selected model. 
                        For best results, create and pass in a `guidance.ChatTemplate` subclass for your model.
  warnings.warn(f"""Chat template {chat_template} was unable to be loaded directly into guidance.


In [7]:
df = pd.read_csv('wikihop_dataset/wikihop-merged-summarized-test.csv')

# select a subset of the queries, just for test:
first_queries = df['query']

# same for correct answers and distractors:
correct_answers = df['answer']
possibilities = df['options']

# and for the sources:
sources = df['sum_supports']

In [43]:
syn_answers = []
for i in range(len(df)):
    syn_answers.append(extract_answer_synthesis(
        synthesisGeneration(
            first_queries[i], possibilities[i], 
            mini['cot'][i], sources[i])))

In [ ]:
ant_answers = []
for i in range(len(df)):
    ant_answers.append(extract_answer_synthesis(
        synthesisGeneration(
            first_queries[i], possibilities[i], 
            mini['antithesis'][i], sources[i])))

In [38]:
df = {
    'query': first_queries,
    'correct': correct_answers,
#    'thesis': mini['thesis'],
#    'pre-antithesis': mini['antithesis'],
#    'antithesis': ant_answers,
#    'pre-synthesis': mini['pre-synthesis'],
    'cot': mini['cot'],
    'synthesis': syn_answers,
    'context': sources
} 

df = pd.DataFrame(df)

In [27]:
def clean_text_final(text):
    text = re.sub(r'[^\w\s.,!?\'"\-:;()]+', '', text)  # Rimuove i caratteri speciali
    text = re.sub(r"['\"-]", '', text)  # Rimuove apostrofi, virgolette e trattini
    text = text.lower()  # Converte in minuscolo
    return text

In [39]:
# Applica la funzione alla colonna 'correct answer'
df['correct'] = df['correct'].apply(clean_text_final)
# df['thesis'] = df['thesis'].apply(clean_text_final)
# df['antithesis'] = df['antithesis'].apply(clean_text_final)
df['synthesis'] = df['synthesis'].apply(clean_text_final)

In [40]:
df.head()

,query,correct,cot,synthesis,context
0,What event did Juan Rossell participate in?,1996 summer olympics,Assistant:The context clearly states that Juan...,1996 summer olympics,"The 2004 Summer Olympic Games, held in Athens..."
1,What languages did John Osteen speak or write?,english,Assistant: The provided context focuses on the...,spanish,Christianity is a monotheistic religion based...
2,What is the parent taxon of Australosuchus?,crocodilia,Assistant:The context states that Australosuch...,crocodile,Mekosuchinae was a subfamily of crocodiles th...
3,What is the'record_label method man' associate...,loud records,Assistant:The context mentions Method Man's co...,1996,"Wu-Tang Clan's debut album, ""Enter the Wu-Tan..."
4,In which administrative territorial entity is ...,raipur district,Assistant:The context clearly states that the ...,raipur district,"Chhattisgarh, a state in central India, is ri..."


In [41]:
df.to_csv('purecot-gemma2b-wikihop.csv')